# Image to Cartoon Converter
### Project: Converting Images to Cartoon Styles Using Computer Vision

This notebook demonstrates 5 different cartoon conversion techniques using OpenCV and image processing algorithms.

---

## 1. Import Required Libraries

In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import zipfile
import os
from io import BytesIO

# Set matplotlib to display images inline
%matplotlib inline

# Configure plot size
plt.rcParams['figure.figsize'] = (20, 8)

## 2. Define Cartoon Conversion Functions

### A. Edge Preserving Filter

In [ ]:
def edge_preserving(img):
    """Edge Preserving Cartoon Effect"""
    # Smooth the image while preserving edges
    smooth = cv2.bilateralFilter(img, 9, 75, 75)
    
    # Convert to grayscale for edge detection
    gray = cv2.cvtColor(smooth, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    
    # Detect edges using adaptive threshold
    edges = cv2.adaptiveThreshold(
        gray, 255, 
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY, 9, 2
    )
    
    # Apply another bilateral filter for color
    color = cv2.bilateralFilter(img, 9, 200, 200)
    
    # Combine edges with color
    return cv2.bitwise_and(color, color, mask=edges)

### B. Color Quantization

In [ ]:
def color_quantization(img):
    """Color Quantization Cartoon Effect"""
    # Reshape image to a list of pixels
    data = np.float32(img).reshape((-1, 3))
    
    # Apply K-means clustering to reduce colors
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)
    _, labels, centers = cv2.kmeans(
        data, 8, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS
    )
    
    # Reconstruct image with quantized colors
    centers = np.uint8(centers)
    quantized = centers[labels.flatten()].reshape(img.shape)
    
    # Edge detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY, 9, 2
    )
    
    # Combine quantized colors with edges
    return cv2.bitwise_and(quantized, quantized, mask=edges)

### C. Stylization

In [ ]:
def stylization(img):
    """Stylization Cartoon Effect"""
    return cv2.stylization(img, sigma_s=150, sigma_r=0.25)

### D. Pencil Sketch

In [ ]:
def pencil_sketch(img):
    """Pencil Sketch Cartoon Effect"""
    _, sketch = cv2.pencilSketch(
        img, 
        sigma_s=60, 
        sigma_r=0.07, 
        shade_factor=0.05
    )
    return sketch

### E. Advanced Cartoon

In [ ]:
def advanced_cartoon(img):
    """Advanced Cartoon Effect"""
    # Multiple passes of bilateral filtering
    smooth = img.copy()
    for _ in range(3):
        smooth = cv2.bilateralFilter(smooth, 9, 75, 75)
    
    # Color quantization with more clusters
    data = np.float32(smooth).reshape((-1, 3))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)
    _, labels, centers = cv2.kmeans(
        data, 12, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS
    )
    
    centers = np.uint8(centers)
    quantized = centers[labels.flatten()].reshape(img.shape)
    
    # Edge detection with larger kernel
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 7)
    edges = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY, 9, 3
    )
    
    return cv2.bitwise_and(quantized, quantized, mask=edges)

## 3. Create Model Dictionary

In [ ]:
MODELS = {
    "Edge Preserving": edge_preserving,
    "Color Quantization": color_quantization,
    "Stylization": stylization,
    "Pencil Sketch": pencil_sketch,
    "Advanced Cartoon": advanced_cartoon
}

print(f"Total Models Available: {len(MODELS)}")
print("Models:", list(MODELS.keys()))

## 4. Load and Process Image

In [ ]:
# ========== CHANGE THIS LINE TO YOUR IMAGE PATH ==========
image_path = 'your_image.jpg'  # ← Put your image filename here
# =========================================================

# Load the image
image = Image.open(image_path).convert("RGB")
original = np.array(image)

# Convert RGB to BGR (OpenCV format)
img_cv = cv2.cvtColor(original, cv2.COLOR_RGB2BGR)

# Display original image
plt.figure(figsize=(8, 6))
plt.imshow(original)
plt.title("Original Image", fontsize=16, fontweight='bold')
plt.axis('off')
plt.show()

print(f"Image loaded successfully!")
print(f"Image size: {original.shape[1]}x{original.shape[0]} pixels")

## 5. Apply All Cartoon Effects

In [ ]:
# Dictionary to store results
results = {"Original": original}

# Apply each cartoon model
print("Processing image with all cartoon models...\n")

for name, func in MODELS.items():
    print(f"Applying {name}...")
    # Apply the cartoon function
    output = func(img_cv)
    # Convert back to RGB for display
    results[name] = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

print("\n✓ All models processed successfully!")

## 6. Display All Results

In [ ]:
# Create subplots for all images
fig, axes = plt.subplots(1, 6, figsize=(24, 6))
fig.suptitle('Image to Cartoon Conversion Results', fontsize=20, fontweight='bold', y=1.02)

# Display each result
for idx, (name, img) in enumerate(results.items()):
    axes[idx].imshow(img)
    axes[idx].set_title(name, fontsize=14, fontweight='bold', pad=10)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 7. Save Individual Cartoon Images

In [ ]:
# Create output folder if it doesn't exist
output_folder = 'cartoon_outputs'
os.makedirs(output_folder, exist_ok=True)

# Save each result
print(f"Saving images to '{output_folder}/' folder...\n")

for name, img in results.items():
    filename = f"{name.lower().replace(' ', '_')}.png"
    filepath = os.path.join(output_folder, filename)
    Image.fromarray(img).save(filepath)
    print(f"✓ Saved: {filename}")

print(f"\n✓ All images saved successfully in '{output_folder}/' folder!")

## 8. Create ZIP Archive

In [ ]:
# Create ZIP file with all cartoon outputs
zip_filename = 'all_cartoon_outputs.zip'

print(f"Creating ZIP archive: {zip_filename}\n")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zip_file:
    for name, img in results.items():
        if name != "Original":  # Skip original image
            # Convert to bytes
            img_buffer = BytesIO()
            Image.fromarray(img).save(img_buffer, format='PNG')
            
            # Add to ZIP
            filename = f"{name.lower().replace(' ', '_')}.png"
            zip_file.writestr(filename, img_buffer.getvalue())
            print(f"✓ Added to ZIP: {filename}")

print(f"\n✓ ZIP archive created: {zip_filename}")
print(f"✓ Total size: {os.path.getsize(zip_filename) / 1024:.2f} KB")

## 9. Summary

This project implements 5 cartoon conversion techniques:
- Edge Preserving: Bilateral filtering + edge detection
- Color Quantization: K-means clustering (8 colors)
- Stylization: OpenCV stylization filter
- Pencil Sketch: Pencil sketch algorithm
- Advanced Cartoon: Multi-pass filtering + 12-cluster K-means

## 10. Display Comparison

In [ ]:
# ========== CUSTOMIZE COMPARISON ==========
model1 = "Edge Preserving"      # ← Change this
model2 = "Advanced Cartoon"     # ← Change this
# ==========================================

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Comparison View', fontsize=18, fontweight='bold')

# Original
axes[0].imshow(results["Original"])
axes[0].set_title("Original", fontsize=14, fontweight='bold')
axes[0].axis('off')

# Model 1
axes[1].imshow(results[model1])
axes[1].set_title(model1, fontsize=14, fontweight='bold')
axes[1].axis('off')

# Model 2
axes[2].imshow(results[model2])
axes[2].set_title(model2, fontsize=14, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()